# **Free-electron bands in a periodic lattice**

<i class="fa fa-home fa-2x"></i><a href="./index.ipynb" style="font-size: 20px"> Go back to index</a>

**Source code:** https://github.com/osscar-org/quantum-mechanics/blob/develop/notebook/free_electron.ipynb

<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

## **Goals**

<p style="text-align: justify;font-size:15px">
The main goal of this notebook is demonstrating the band structures and density of states for free-electron model in a periodic lattice.
</p>

<details close>
    <summary style="font-size: 20px"><b>Sub-goals</b></summary>
    <ol style="text-align: justify;font-size:15px">
        <li> Understand the free-electron model. </li>
        <li> Examine the electronic band structure for different cells. </li>
        <li> Know how to construct the Brillouin zone. </li>
        <li> Understand Monkhorst-Pack k-points. </li>
        <li> Examine the density of states. </li>
        <li> Describe how to calculate the density of states. </li>
    </ol>

</details>

## **Background theory**

<p style="text-align: justify;font-size:15px">    
    Here, we employ the empty lattice approximation for the electrons in a periodic 
    solid system.  It computes and plots the electronic band structure for three 
    type of cells (simple cubic, FCC and BCC). We get the path of the band structure 
    from the <a href="https://seekpath.readthedocs.io/en/latest/index.html">seekpath</a>
    package. We demonstrate three different methods to calculate the corresponding 
    density of states (DOS). These are the linear tetrahedron interpolation (LTI), 
    simple histogram and Gaussian smearing methods.
</p>

<details close>
<summary style="font-size: 20px">Empty lattice approximation</summary>
<p style="text-align: justify;font-size:15px"> 
    In the empty lattice approximation, the electrons move "freely" in the 
    periodic potential. There is no electron-electron interaction. 
</p>
    
<p style="text-align: justify;font-size:15px">
    The eigenfunctions of the Schrödinger equation for free electrons are:
</p>

<p style="text-align: center;font-size:15px">
$\large \psi(\vec{r}) = e^{i\vec{k} \vec{r}}$   
</p>
    
<p style="text-align: justify;font-size:15px">
    When the $\vec{k'}$ is outside the 1st Brillouin zone, the plane wave can be written as:
</p>
    
<p style="text-align: center;font-size:15px">
$\large \psi(\vec{r}) = e^{i\vec{k} \vec{r}}e^{i\vec{G} \vec{r}} = e^{i(\vec{k}+\vec{G})\vec{r}}$   
</p>
    
<p style="text-align: justify;font-size:15px">
    where, $\vec{k}$ vector is inside the first brillouin zone and $\vec{G}$ is the reciprocal
    lattice vector. The dispersion is:
</p>
    
<p style="text-align: center;font-size:15px">
    $\large E = \frac{\hbar^2(\vec{k}+\vec{G})^2}{2m}$
</p>

    
<p style="text-align: justify;font-size:15px">
    Please read more at the Wikipedia:
    <a href="https://en.wikipedia.org/wiki/Empty_lattice_approximation#:~:text=The%20empty%20lattice%20approximation%20describes,the%20energy%20of%20free%20electrons">Empty lattice approximation</a>
</p>
          
</details>

<details close>
<summary style="font-size: 20px">Electronic band structure</summary>
<p style="text-align: justify;font-size:15px">
    In the solid-state physics, the electronic band structure of 
    a solid describes the quantum-mechanical behavior of electrons.
</p>

<p style="text-align: justify;font-size:15px"> 
    "Band theory derives these bands and band gaps by examining the allowed quantum mechanical 
    wave functions for an electron in a large, periodic lattice of atoms or molecules. Band theory
    has been successfully used to explain many physical properties of solids, such as electrical
    resistivity and optical absorption, and forms the foundation of the understanding of all 
    solid-state devices (transistors, solar cells, etc.)." - from Wikipedia.
</p>
  
<image src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/ef/Solid_state_electronic_band_structure.svg/880px-Solid_state_electronic_band_structure.svg.png" width="500"></image>
    
<p style="text-align: justify;font-size:15px">
    Molecular diagrams can present the discrete energy levels for the molecular systems. 
    In contrast, solid system always have a very large number of the orbitals. 
    It leads to the energy levels to close together. Hence, the energy levels in solid 
    are considered as continuum energy bands.
</p>
    
<p style="text-align: justify;font-size:15px"> 
    Since the wavevector is in three dimensions ($k_x$, $k_y$ and $k_z$), it is difficult to plot
    the bands as a function of the wavevector (4 dimensional plotting). Usually, 
    the bands are plotted along the straight lines, which connects high symmetry points.
</p>
    
<image src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/c1/Brillouin_Zone_%281st%2C_FCC%29.svg/440px-Brillouin_Zone_%281st%2C_FCC%29.svg.png" width="300"></image>
    
</details>

## **Tasks and exercises**

<ol style="text-align: justify;font-size:15px">
    <li> How to construct the 1st Brillouin zone?
    <details style="color: blue">
    <summary>Hints</summary>
    The Brillouin zone is built from the reciprocal space.
    </details>
    </li>
    <li> Can you describe the shape of the band structure in the 1st Brillouin zone?
    <details style="color: blue">
    <summary>Hints</summary>
    In the free electron model, the dispersion is: $E=\frac{\hbar^2k^2}{2m}$
    </details>   
    </li>
    <li> 
    Increase the number of kpoints with the slider. Compute the DOS with 
    all the three methods. How is the density of the states changing with 
    increasing the number of kpoints?
    <details style="color: blue">
    <summary>Hints</summary>
    The blue line shows the analytical solution for the DOS. Do the calculated results
        imporve with more kpoints?
    </details>   
    </li>
    <li> Calculate the DOS with different G vector ranges. How does the DOS change with the G
        vector ranges? Can you explain it?
    <details style="color: blue">
    <summary>Hints</summary>
    Read the "free electron model" section. In the periodic system, the dispersion is:
        $E=\frac{\hbar^2}{2m}(k+G_n)^2$
    </details>   
    </li>
    <li> Which method gives most accurate results? Which method is fastest and why?
    <details style="color: blue">
    <summary>Hints</summary>
    Read the "density of states" section and understand how the DOS is calculated with different methods.
    </details>   
    </li>
</ol>

<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

In [ ]:
import numpy as np
import seekpath
import re
import matplotlib
from ase.dft.dos import linear_tetrahedron_integration as lti
from ase.dft.kpoints import monkhorst_pack
from ase.cell import Cell
from scipy.stats import multivariate_normal
from widget_bzvisualizer import BZVisualizer

In [ ]:
def prettify(label):
    """
    Prettifier for matplotlib, using LaTeX syntax
    :param label: a string to prettify
    """

    label = (
        label
            .replace('GAMMA', r'$\Gamma$')
            .replace('DELTA', r'$\Delta$')
            .replace('LAMBDA', r'$\Lambda$')
            .replace('SIGMA', r'$\Sigma$')
    )
    label = re.sub(r'_(.?)', r'$_{\1}$', label)

    return label

In [ ]:
def _get_band_energies(kpoints_list, b1, b2, b3, g_vectors_range):
    energy_data_curves = np.zeros(((2*g_vectors_range+1)**3, len(kpoints_list)), dtype=np.float_)

    cnt = 0
    for g_i in range(-g_vectors_range,g_vectors_range+1):
        for g_j in range(-g_vectors_range,g_vectors_range+1):
            for g_k in range(-g_vectors_range,g_vectors_range+1):
                g_vector = b1 * g_i + b2*g_j + b3 * g_k
                energy_data_curves[cnt] = np.sum(0.5*(kpoints_list + g_vector)**2, axis=1)# This is k^2 - NOTE: units to be double checked!
                cnt += 1


    # bands are ordered as follows: first band, second band, ...
    return energy_data_curves

def _compute_dos(kpts, G, ranges):
    eigs = []
    n = ranges
    
    for i in range(-n, n+1):
        for j in range(-n, n+1):
            for k in range(-n, n+1):
                g_vector = i*G[0] + j*G[1] + k*G[2]
                eigs.append(np.sum(0.5*(kpts + g_vector)**2, axis=3))

    eigs = np.moveaxis(eigs, 0, -1)
    return eigs
   
def _compute_total_kpts(kpts, G, ranges):
    tot_kpts = []
    n = ranges

    for i in range(-n, n+1):
        for j in range(-n, n+1):
            for k in range(-n, n+1):
                g_vector = i*G[0] + j*G[1] + k*G[2]
                tot_kpts.extend(kpts+g_vector)
    return np.array(tot_kpts)
    

In [ ]:
def get_bands(real_lattice_bohr, reference_distance = 0.025, g_vectors_range = 3):

    # Simple way to get automatically the band path:
    # I go back to real space, just put a single atom at the origin,
    # then go back with seekpath.
    # NOTE! This might not give the most general path, as e.g. there are two
    # options for cubic FCC (cF1 and cF2 in seekpath).
    # But this should be general enough for this tool.

    structure = (real_lattice_bohr, [[0., 0., 0.]], [1])
    # Use a H atom at the origin
    seekpath_path = seekpath.get_explicit_k_path(structure, reference_distance=reference_distance)
    b1, b2, b3 = np.array(seekpath_path['reciprocal_primitive_lattice'])

    all_kpoints_x = np.array(seekpath_path['explicit_kpoints_linearcoord'])
    all_kpoints_list = np.array(seekpath_path['explicit_kpoints_abs'])

    segments_data = []
    for segment_indices in seekpath_path['explicit_segments']:
        start_label = seekpath_path['explicit_kpoints_labels'][segment_indices[0]]
        end_label = seekpath_path['explicit_kpoints_labels'][segment_indices[1]-1]

        kpoints_x = all_kpoints_x[slice(*segment_indices)]
        kpoints_list = all_kpoints_list[slice(*segment_indices)]

        energy_bands = _get_band_energies(kpoints_list, b1, b2, b3, g_vectors_range)

        segments_data.append({
            'start_label': start_label,
            'end_label': end_label,
            'kpoints_list': kpoints_list,
            'kpoints_x': kpoints_x,
            'energy_bands': energy_bands,
            'b1': b1,
            'b2': b2,
            'b3': b3,
        })

    return segments_data

In [ ]:
%matplotlib widget

import time
import matplotlib.pyplot as plt
from ipywidgets import Output, Button, RadioButtons, IntSlider, HBox, VBox, Checkbox, Label, FloatSlider

alat_bohr = 7.72

lattices = np.zeros((3, 3, 3));

lattices[0] = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]]) * alat_bohr / 2.0;
lattices[1] = np.array([[0, 1, 1], [1, 0, 1], [1, 1, 0]]) * alat_bohr / 2.0;
lattices[2] = np.array([[-1, 1, 1], [1, -1, 1], [1, 1, -1]]) * alat_bohr / 2.0;

real_lattice_bohr = lattices[0]
bz = BZVisualizer(real_lattice_bohr, [[0.0, 0.0, 0.0]], [1], True, height='400px')

In [ ]:
#G = Cell(real_lattice_bohr).reciprocal()*2*np.pi

style = {'description_width': 'initial'}

output = Output()
cell_type = RadioButtons(options=['Simple cubic', 'FCC', 'BCC'], value='Simple cubic', description="Cell type:")
nkpt = IntSlider(value=4, min=4, max=11, description="Number of kpoint:", style=style)
grange = IntSlider(value=0, min=0, max=3, description="Gvector range:", style=style)
gcov = FloatSlider(value=0.5, min=0.1, max=1.0, description="Guassian covariance:", style=style)

def on_celltype_changed(c):
    global real_lattice_bohr
    real_lattice_bohr = lattices[cell_type.index]
    ax.clear()

    plot_bandstructure('bands')
    bz.cell = real_lattice_bohr.tolist()

cell_type.observe(on_celltype_changed, names='value');


def plot_bandstructure(c):
    global G, segments_data, lbands
    
    segments_data = get_bands(real_lattice_bohr)
    G = np.array([segments_data[0]['b1'], segments_data[0]['b2'], segments_data[0]['b3']])
    
    x_ticks = []
    x_labels = []
    lbands = []

    for segment_data in segments_data:
        if not x_labels:
            x_labels.append(prettify(segment_data['start_label']))
            x_ticks.append(segment_data['kpoints_x'][0])
        else:
            if x_labels[-1] != prettify(segment_data['start_label']):
                x_labels[-1] += "|" + prettify(segment_data['start_label'])
        x_labels.append(prettify(segment_data['end_label']))
        x_ticks.append(segment_data['kpoints_x'][-1])

        for energy_band in segment_data['energy_bands']:
            line, = ax.plot(segment_data['kpoints_x'], energy_band, 'k')
            lbands.append(line)

    ax.set_ylim([0, 5])
    ax.yaxis.tick_right()
    ax.yaxis.set_label_position("right")
    ax.set_ylabel('Free-electron energy (eV)')
    ax.set_xlim([np.min(x_ticks), np.max(x_ticks)])
    ax.set_xticks(x_ticks)
    ax.set_xticklabels(x_labels)
    ax.grid(axis='x', color='red', linestyle='-', linewidth=0.5)
    fig.tight_layout()
    
    update_bands_color('bands')
    
def update_bands_color(c):
    n = 3
    
    shape = (nkpt.value, nkpt.value, nkpt.value)
    kpts = np.dot(monkhorst_pack(shape), G).reshape(shape + (3,))
    eigs = _compute_dos(kpts, G, grange.value)

    index = 0
    
    for segment_data in segments_data:
        for i in range(-n, n+1):
            for j in range(-n, n+1):
                for k in range(-n, n+1): 
                    if abs(i) <= grange.value and abs(j) <= grange.value and abs(k) <=grange.value:
                        lbands[index].set_color('r')
                    else:
                        lbands[index].set_color('k')
                    index+=1

grange.observe(update_bands_color, names="value")

    
with output:
    global fig, ax
    fig, ax = plt.subplots()
    fig.set_size_inches(3.7, 5.0)
    fig.canvas.header_visible = False
    fig.canvas.layout.width = "430px"
    plot_bandstructure('bands')

    
label1 = Label(value="Compute DOS by different methods:")
label2 = Label(value="(the number of kpoints in all three dimensions)")
label3 = Label(value="(the number of G vector ranges in all three dimensions)")

In [ ]:
display(HBox([VBox([bz,cell_type]), output]))

<details>
    <summary style="font-size: 22px;"><b>Legend</b></summary>

<p style="text-align: justify;font-size:15px">
    The 1st Brillouin zone of the selected cell is shown on the top. 
    The path for the band structure is shown with blue vectors and 
    kpoints are red dots in the Brillouin zone.
    
    The figure on the bottom left shows the calculated band structure. 
    The figure on the bottom right shows the corresponding density of states (DOS).
    
    We provide three kinds of cell structure, which are simple cubic, 
    face centre cubic (FCC) and body centre cubic (BCC).  Use the radio 
    button to select the cell type. The number of kpoints and G vector ranges 
    can be tuned with the sliders.
</p>
    
<p style="text-align: justify;font-size:15px">
    Click the button to compute the DOS with selected method. The DOS results
    will appear in the figure on the bottom right. It will take some minutes 
    to get the results with many kpoints and large G vector ranges. For the 
    Gaussian smearing method, you can tune the covariance for the Gaussian 
    functions with the slider next to the "Gaussian" button.
</p>